# 04. スペクトログラム基礎

スペクトログラム（時間-周波数表示）の基本概念を学び、音の変化を視覚的に理解します。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'DejaVu Sans'

## 1. 基本的なスペクトログラム

In [ ]:
# 周波数が変化する信号の生成
def create_chirp_signal(duration, sample_rate, f_start, f_end):
    """
    チャープ信号（周波数が線形に変化）を生成
    """
    t = np.linspace(0, duration, int(sample_rate * duration), False)
    # 線形チャープ
    chirp = signal.chirp(t, f_start, duration, f_end, method='linear')
    return t, chirp

# パラメータ設定
sample_rate = 8000  # サンプリングレート（低めに設定）
duration = 3.0
f_start = 100   # 開始周波数 100Hz
f_end = 2000    # 終了周波数 2000Hz

# チャープ信号の生成
t, chirp_signal = create_chirp_signal(duration, sample_rate, f_start, f_end)

# スペクトログラムの計算
frequencies, times, Sxx = signal.spectrogram(chirp_signal, sample_rate, 
                                           window='hann', nperseg=256, 
                                           noverlap=128)

plt.figure(figsize=(15, 10))

# 元の信号
plt.subplot(2, 1, 1)
plt.plot(t, chirp_signal)
plt.title('チャープ信号（100Hz → 2000Hz）')
plt.xlabel('時間 (秒)')
plt.ylabel('振幅')
plt.grid(True)

# スペクトログラム
plt.subplot(2, 1, 2)
plt.pcolormesh(times, frequencies, 10 * np.log10(Sxx), 
               shading='gouraud', cmap='viridis')
plt.title('スペクトログラム')
plt.xlabel('時間 (秒)')
plt.ylabel('周波数 (Hz)')
plt.colorbar(label='パワー (dB)')

plt.tight_layout()
plt.show()

## 2. 複数の周波数成分を持つ信号

In [ ]:
# 複雑な音響信号の生成
def create_complex_signal(duration, sample_rate):
    t = np.linspace(0, duration, int(sample_rate * duration), False)
    
    # 基音とハーモニクス
    fundamental = 220  # Hz (A3)
    signal_complex = np.zeros_like(t)
    
    # セクション1: 基音のみ
    mask1 = (t >= 0) & (t < 1)
    signal_complex[mask1] = np.sin(2 * np.pi * fundamental * t[mask1])
    
    # セクション2: 基音 + 2倍音
    mask2 = (t >= 1) & (t < 2)
    signal_complex[mask2] = (np.sin(2 * np.pi * fundamental * t[mask2]) + 
                           0.5 * np.sin(2 * np.pi * 2 * fundamental * t[mask2]))
    
    # セクション3: 基音 + 2倍音 + 3倍音
    mask3 = (t >= 2) & (t < 3)
    signal_complex[mask3] = (np.sin(2 * np.pi * fundamental * t[mask3]) + 
                           0.5 * np.sin(2 * np.pi * 2 * fundamental * t[mask3]) +
                           0.3 * np.sin(2 * np.pi * 3 * fundamental * t[mask3]))
    
    # セクション4: コード（複数の基音）
    mask4 = (t >= 3) & (t < 4)
    signal_complex[mask4] = (np.sin(2 * np.pi * 220 * t[mask4]) +      # A
                           np.sin(2 * np.pi * 277.18 * t[mask4]) +    # C#
                           np.sin(2 * np.pi * 329.63 * t[mask4]))     # E
    
    return t, signal_complex

# 複雑な信号の生成
t_complex, complex_signal = create_complex_signal(4.0, sample_rate)

# スペクトログラムの計算
freq_comp, time_comp, Sxx_comp = signal.spectrogram(complex_signal, sample_rate,
                                                   window='hann', nperseg=512,
                                                   noverlap=256)

plt.figure(figsize=(15, 12))

# 波形
plt.subplot(3, 1, 1)
plt.plot(t_complex, complex_signal)
plt.title('複雑な音響信号')
plt.ylabel('振幅')
plt.grid(True)
plt.axvline(x=1, color='r', linestyle='--', alpha=0.7)
plt.axvline(x=2, color='r', linestyle='--', alpha=0.7)
plt.axvline(x=3, color='r', linestyle='--', alpha=0.7)

# 波形の拡大表示
plt.subplot(3, 1, 2)
zoom_start = int(2.5 * sample_rate)  # 2.5秒から
zoom_end = int(3.0 * sample_rate)    # 3.0秒まで
plt.plot(t_complex[zoom_start:zoom_end], complex_signal[zoom_start:zoom_end])
plt.title('波形拡大表示（2.5-3.0秒: 基音+2倍音+3倍音）')
plt.ylabel('振幅')
plt.grid(True)

# スペクトログラム
plt.subplot(3, 1, 3)
plt.pcolormesh(time_comp, freq_comp, 10 * np.log10(Sxx_comp + 1e-10),
               shading='gouraud', cmap='plasma')
plt.title('スペクトログラム（音響成分の変化）')
plt.xlabel('時間 (秒)')
plt.ylabel('周波数 (Hz)')
plt.colorbar(label='パワー (dB)')
plt.ylim(0, 1500)  # 表示周波数範囲を制限

# セクション境界線
plt.axvline(x=1, color='white', linestyle='--', alpha=0.8)
plt.axvline(x=2, color='white', linestyle='--', alpha=0.8)
plt.axvline(x=3, color='white', linestyle='--', alpha=0.8)

# セクションラベル
plt.text(0.5, 1300, '基音のみ', color='white', ha='center', fontweight='bold')
plt.text(1.5, 1300, '基音+2倍音', color='white', ha='center', fontweight='bold')
plt.text(2.5, 1300, '基音+2,3倍音', color='white', ha='center', fontweight='bold')
plt.text(3.5, 1300, 'コード', color='white', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

## 3. 窓関数の影響

In [ ]:
# 短いトーンバースト信号
def create_tone_burst(duration, sample_rate, frequency, burst_duration=0.1):
    t = np.linspace(0, duration, int(sample_rate * duration), False)
    signal = np.zeros_like(t)
    
    # 0.5秒から短いバーストを配置
    burst_start = int(0.5 * sample_rate)
    burst_samples = int(burst_duration * sample_rate)
    burst_end = burst_start + burst_samples
    
    # バースト信号（ハニング窓でエンベロープ）
    burst_t = np.linspace(0, burst_duration, burst_samples)
    envelope = np.hanning(burst_samples)
    burst_signal = envelope * np.sin(2 * np.pi * frequency * burst_t)
    
    signal[burst_start:burst_end] = burst_signal
    
    return t, signal

# トーンバースト信号の生成
t_burst, burst_signal = create_tone_burst(2.0, sample_rate, 800, 0.1)

# 異なる窓関数でスペクトログラムを計算
windows = ['hann', 'hamming', 'blackman', 'boxcar']
nperseg_values = [128, 256, 512]  # 窓長

plt.figure(figsize=(20, 15))

# 元の信号
plt.subplot(len(windows) + 1, len(nperseg_values), 1)
plt.plot(t_burst, burst_signal)
plt.title('トーンバースト信号')
plt.ylabel('振幅')
plt.grid(True)

plot_idx = len(nperseg_values) + 1

for i, window_type in enumerate(windows):
    for j, nperseg in enumerate(nperseg_values):
        freq_win, time_win, Sxx_win = signal.spectrogram(
            burst_signal, sample_rate, 
            window=window_type, nperseg=nperseg, 
            noverlap=nperseg//2
        )
        
        plt.subplot(len(windows) + 1, len(nperseg_values), plot_idx)
        plt.pcolormesh(time_win, freq_win, 10 * np.log10(Sxx_win + 1e-10),
                      shading='gouraud', cmap='viridis')
        plt.title(f'{window_type}窓, 窓長={nperseg}')
        if j == 0:
            plt.ylabel('周波数 (Hz)')
        if i == len(windows) - 1:
            plt.xlabel('時間 (秒)')
        plt.ylim(0, 2000)
        
        plot_idx += 1

plt.tight_layout()
plt.show()

# 窓関数の形状を比較
plt.figure(figsize=(12, 8))
window_length = 256

for i, window_type in enumerate(windows):
    if window_type == 'boxcar':
        window_func = np.ones(window_length)
    else:
        window_func = signal.get_window(window_type, window_length)
    
    plt.subplot(2, 2, i+1)
    plt.plot(window_func)
    plt.title(f'{window_type.capitalize()}窓')
    plt.ylabel('振幅')
    plt.grid(True)
    if i >= 2:
        plt.xlabel('サンプル')

plt.tight_layout()
plt.show()

## 4. 時間-周波数分解能のトレードオフ

In [ ]:
# 近接した周波数と短い時間変化の信号
def create_resolution_test_signal(sample_rate):
    duration = 2.0
    t = np.linspace(0, duration, int(sample_rate * duration), False)
    signal = np.zeros_like(t)
    
    # セクション1: 近接した2つの周波数（周波数分解能テスト）
    mask1 = (t >= 0.2) & (t < 0.8)
    signal[mask1] = (np.sin(2 * np.pi * 500 * t[mask1]) + 
                    np.sin(2 * np.pi * 520 * t[mask1]))  # 20Hz差
    
    # セクション2: 短い時間の周波数変化（時間分解能テスト）
    # 1000Hz → 1500Hz → 1000Hz （各0.1秒）
    burst_duration = 0.1
    for i, freq in enumerate([1000, 1500, 1000]):
        start_time = 1.0 + i * burst_duration
        end_time = start_time + burst_duration
        mask = (t >= start_time) & (t < end_time)
        envelope = np.hanning(np.sum(mask))
        signal[mask] = envelope * np.sin(2 * np.pi * freq * t[mask])
    
    return t, signal

t_res, res_signal = create_resolution_test_signal(sample_rate)

# 異なる窓長での比較
window_sizes = [64, 256, 1024]
window_titles = ['短い窓(高時間分解能)', '中程度の窓', '長い窓(高周波数分解能)']

plt.figure(figsize=(18, 12))

# 元の信号
plt.subplot(len(window_sizes) + 1, 1, 1)
plt.plot(t_res, res_signal)
plt.title('分解能テスト信号')
plt.ylabel('振幅')
plt.grid(True)
plt.axvline(x=0.2, color='r', linestyle='--', alpha=0.5)
plt.axvline(x=0.8, color='r', linestyle='--', alpha=0.5)
plt.axvline(x=1.0, color='g', linestyle='--', alpha=0.5)
plt.axvline(x=1.3, color='g', linestyle='--', alpha=0.5)

for i, (window_size, title) in enumerate(zip(window_sizes, window_titles)):
    freq_res, time_res, Sxx_res = signal.spectrogram(
        res_signal, sample_rate,
        window='hann', nperseg=window_size,
        noverlap=window_size//2
    )
    
    plt.subplot(len(window_sizes) + 1, 1, i + 2)
    plt.pcolormesh(time_res, freq_res, 10 * np.log10(Sxx_res + 1e-10),
                  shading='gouraud', cmap='hot')
    plt.title(f'{title} (窓長={window_size}サンプル)')
    plt.ylabel('周波数 (Hz)')
    plt.ylim(0, 2000)
    
    # 周波数分解能の表示
    freq_resolution = sample_rate / window_size
    time_resolution = window_size / sample_rate * 1000  # ms
    
    plt.text(0.05, 1800, f'周波数分解能: {freq_resolution:.1f} Hz\n時間分解能: {time_resolution:.1f} ms',
            bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8),
            fontsize=10)
    
    if i == len(window_sizes) - 1:
        plt.xlabel('時間 (秒)')
        plt.colorbar(label='パワー (dB)')

plt.tight_layout()
plt.show()

print("分解能の比較:")
print("窓長\t周波数分解能(Hz)\t時間分解能(ms)")
for window_size in window_sizes:
    freq_res = sample_rate / window_size
    time_res = window_size / sample_rate * 1000
    print(f"{window_size}\t{freq_res:.1f}\t\t\t{time_res:.1f}")

## 練習問題

1. ピアノの音（複数のハーモニクスを含む）をシミュレートしてスペクトログラムを作成してみましょう
2. 音楽のメロディー（複数の音符の連続）のスペクトログラムを作成してみましょう
3. ノイズが混入した信号のスペクトログラムを作成し、ノイズの影響を観察してみましょう